In [ ]:
import sirf.STIR as pet
import matplotlib.pyplot as plt
from ipywidgets import interact
# Siemens_mMR_NEMA_IQ NeuroLF_Hoffman_Dataset
kappa = pet.ImageData("/workspaces/PETRIC-UCL-EWS/data/Siemens_mMR_NEMA_IQ/kappa.hv")#.get_uniform_copy(1)
img = pet.ImageData("/workspaces/PETRIC-UCL-EWS/data/Siemens_mMR_NEMA_IQ/PETRIC/reference_image.hv")

pet.AcquisitionData.set_storage_scheme('memory') # needed for get_subsets()
penalty_strength = 1 / 700 # default choice
prior = pet.CudaRelativeDifferencePrior()
epsilon = img.max() * 1e-3
prior.set_epsilon(epsilon)
prior.set_penalisation_factor(penalty_strength)
prior.set_kappa(kappa)
prior.set_up(img)
trunc_cyl = pet.TruncateToCylinderProcessor()
img_grad = prior.get_gradient(img)
trunc_cyl.apply(img_grad)

# interactive plot of kappa and img

def plot_slice(slice_n):
    fig, ax = plt.subplots(1, 3, figsize=(15, 5))
    fig.colorbar(ax[0].imshow(kappa.as_array()[slice_n,:,:]), ax=ax[0])
    fig.colorbar(ax[1].imshow(img.as_array()[slice_n,:,:]), ax=ax[1])
    fig.colorbar(ax[2].imshow(img_grad.as_array()[slice_n,:,:]), ax=ax[2])
    ax[0].set_title('kappa')
    ax[1].set_title('img')
    ax[2].set_title('img_grad')
    plt.show()

interact(plot_slice, slice_n=(0, kappa.dimensions()[0]-1))


In [ ]:
import numpy as np
# compute weights 1/euclidean norm
cp_weights = np.zeros([3,3,3])
voxel_sizes = img.voxel_sizes()
for i in range(3):
    for j in range(3):
        for k in range(3):
            cp_weights[i,j,k] = voxel_sizes[2]/np.sqrt(((i-1)*voxel_sizes[0])**2 + ((j-1)*voxel_sizes[1])**2 + ((k-1)*voxel_sizes[2])**2)
cp_weights[1,1,1] = 0
cp_image = np.asarray(img.as_array(), dtype=np.float32)
z_dim, y_dim, x_dim = cp_image.shape
cp_penalisation_factor = np.ones(1, dtype=np.float32)*penalty_strength
cp_gamma = np.ones(1, dtype=np.float32)*2
cp_epsilon = prior.get_epsilon()
cp_kappa = np.asarray(kappa.as_array(), dtype=np.float32)
cp_weights = np.asarray(cp_weights, dtype=np.float32)



def get_rdpz_grad(x, kappa, epsilon, weights, penalty_strength):
    x_padded = np.pad(x, pad_width=((1, 1), (1, 1), (1, 1)))
    kappa_padded = np.pad(kappa, pad_width=((1, 1), (1, 1), (1, 1))) # , mode='edge'
    x_grad = np.zeros_like(x)
    z_dim, y_dim, x_dim = x.shape
    for dz in [0, 1, 2]:
        for dy in [0, 1, 2]:
            for dx in [0, 1, 2]:
                x_neighbour = x_padded[dz:dz+z_dim, dy:dy+y_dim, dx:dx+x_dim]
                kappa_neighbour = kappa_padded[dz:dz+z_dim, dy:dy+y_dim, dx:dx+x_dim]
                difference = x - x_neighbour
                kappa_val = kappa*kappa_neighbour
                numerator = (difference)*(2 * np.abs(difference) + x + 3 * x_neighbour)
                denominator = (x + x_neighbour + 2 * np.abs(difference) + epsilon)**2
                x_grad += weights[dz,dy,dx] * penalty_strength * kappa_val * numerator / denominator
    return x_grad

cp_image_grad = get_rdpz_grad(cp_image, cp_kappa, cp_epsilon, cp_weights, penalty_strength)

test = img_grad.clone().fill(cp_image_grad)
trunc_cyl.apply(test)

def plot_slice(slice_n):
    fig, ax = plt.subplots(1, 3, figsize=(15, 5))
    max_val = np.max([img_grad.as_array()[slice_n,:,:], cp_image_grad[slice_n,:,:]])
    min_val = np.min([img_grad.as_array()[slice_n,:,:], cp_image_grad[slice_n,:,:]])
    fig.colorbar(ax[0].imshow(img_grad.as_array()[slice_n,:,:], vmax = max_val, vmin = min_val), ax=ax[0])
    fig.colorbar(ax[1].imshow(test.as_array()[slice_n,:,:], vmax = max_val, vmin = min_val), ax=ax[1])
    fig.colorbar(ax[2].imshow(img_grad.as_array()[slice_n,:,:] - test.as_array()[slice_n,:,:]), ax=ax[2])
    ax[0].set_title('img_grad')
    ax[1].set_title('cp_image_grad')
    plt.show()

interact(plot_slice, slice_n=(0, kappa.dimensions()[0]-1))